# IMDB movie review sentiment classification using huggingface models

In this notebook, we'll test pre-trained sentiment analysis models and later finetune a DistilBERT model to perform IMDB movie review sentiment classification. This notebook is adpated from [Getting Started with Sentiment Analysis using Python](https://huggingface.co/blog/sentiment-analysis-python).

First, we need to install several packages. 

In [1]:
!pip install datasets transformers huggingface_hub

Defaulting to user installation because normal site-packages is not writeable


## Import the libraries

In [2]:
from transformers import pipeline
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
import numpy as np
from datasets import load_metric
from huggingface_hub import notebook_login
from transformers import TrainingArguments, Trainer
from transformers import pipeline

## Check if torch is using the GPU

In [3]:
print('Using PyTorch version:', torch.__version__)
if torch.cuda.is_available():
    print('Using GPU, device name:', torch.cuda.get_device_name(0))
    device = torch.device('cuda')
else:
    print('No GPU found, using CPU instead.') 
    device = torch.device('cpu')

Using PyTorch version: 2.4.1+rocm6.1
Using GPU, device name: AMD Instinct MI250X


## Use Pre-trained Sentiment Analysis Models

In [4]:
sentiment_pipeline = pipeline("sentiment-analysis",device=device)
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

- This code snippet above utilizes the **[pipeline](https://huggingface.co/docs/transformers/main_classes/pipelines)** class to generate predictions using models from the Hub. It applies the [default sentiment analysis model](https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english?text=I+like+you.+I+love+you) to evaluate the provided list of text data.
- The analysis results are **POSITIVE** for first entry and **NEGATIVE** for the second entry.

### One can also use a specific sentiment analysis model by providing the name of the model, e.g., if you want a sentiment analysis model for tweets, you can specify the model id.

In [5]:
specific_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis", device = device)
specific_model(data)

[{'label': 'POS', 'score': 0.9916695356369019},
 {'label': 'NEG', 'score': 0.9806600213050842}]

## Fine-tuning DistilBERT model using IMDB dataset 

- The [IMDB](https://huggingface.co/datasets/stanfordnlp/imdb) contains 50000 movies reviews from the Internet Movie Database, split into 25000 reviews for training and 25000 reviews for testing. Half of the reviews are positive and half are negative. 

- IMDB dataset is relatively large, so let's use 5000 samples for training to speed up our process for this exercise.

In [6]:
imdb = load_dataset("imdb")
small_train_dataset = imdb["train"].shuffle(seed=0).select([i for i in list(range(5000))])
test_dataset = imdb["test"]

To preprocess our data, we will use DistilBERT tokenizer:

In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

- Next, we will prepare the text inputs for the model for both splits of our dataset (training and test) by using the map method:

In [8]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)
 
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

- To speed up training, let's use a data_collator to convert your training samples to PyTorch tensors and concatenate them with the correct amount of padding:

In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Training the model
- We will be throwing away the pretraining head of the DistilBERT model and replacing it with a classification head fine-tuned for sentiment analysis. This enables us to transfer the knowledge from DistilBERT to our custom model.

In [10]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


- Then, let's define the metrics you will be using to evaluate how good is your fine-tuned model (accuracy and f1 score)

In [11]:
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

- Define the training arguments

In [12]:
repo_name = "finetuning-sentiment-model-5000-samples"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=False,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

- Start training

In [13]:
trainer.train()

[2024-09-26 10:21:48,631] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Step,Training Loss
500,0.283400


TrainOutput(global_step=626, training_loss=0.2560590951206585, metrics={'train_runtime': 92.9765, 'train_samples_per_second': 107.554, 'train_steps_per_second': 6.733, 'total_flos': 1313372861612160.0, 'train_loss': 0.2560590951206585, 'epoch': 2.0})

- Evaluate the model

In [14]:
trainer.evaluate()

/tmp/ipykernel_81323/4164298400.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 0.26140642166137695,
 'eval_accuracy': 0.9116,
 'eval_f1': 0.9127723397537101,
 'eval_runtime': 76.5547,
 'eval_samples_per_second': 326.564,
 'eval_steps_per_second': 20.417,
 'epoch': 2.0}

- Model inference

In [15]:
pipe = pipeline("text-classification", model=model,tokenizer=tokenizer, device = device)
pipe(["I love this move", "This movie sucks!"])

[{'label': 'LABEL_1', 'score': 0.980747401714325},
 {'label': 'LABEL_0', 'score': 0.9798480868339539}]

## Task 1 Run this script with GPU

## Task 2 Compare the test dataset accuracy achieved from finetuned DistilBERT model and the previous RNN model. What do you notice?